In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd drive/MyDrive/Sapienza/DL

/content/drive/MyDrive/Sapienza/DL


In [3]:
import pickle
import pandas as pd
import numpy as np
import json
import itertools
import gensim
path="SEMEVAL-2021-task6-corpus/data/"


In [4]:
with open('original_training_task2_labels1.pkl', 'rb') as handle:
    train_labs1 = pickle.load(handle)

with open('original_training_task2_labels2.pkl', 'rb') as handle:
    train_labs2 = pickle.load(handle)

with open('original_training_task2_features.pkl', 'rb') as handle:
    train_feats = pickle.load(handle)

with open('task2_all_labels.pkl', 'rb') as handle:
    all_labels = pickle.load(handle)

with open('task2_labels_dict.pkl', 'rb') as handle:
    labs_dict = pickle.load(handle)

with open('original_dev_task2_labels1.pkl', 'rb') as handle:
    dev_labs1 = pickle.load(handle)

with open('original_dev_task2_labels2.pkl', 'rb') as handle:
    dev_labs2 = pickle.load(handle)

with open('original_dev_task2_features.pkl', 'rb') as handle:
    dev_feats = pickle.load(handle)



In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(train_feats)
X_train_counts.shape

(1641, 3139)

In [6]:
tfidf_transformer = TfidfTransformer()
X_train = tfidf_transformer.fit_transform(X_train_counts)
X_train.shape

(1641, 3139)

In [7]:
dev_counts=count_vect.transform(dev_feats)
X_dev=tfidf_transformer.transform(dev_counts)
X_dev.shape

(197, 3139)

In [8]:
X_train=X_train.toarray()
X_dev=X_dev.toarray()

In [9]:
X_train=np.expand_dims(X_train,axis=2)
X_dev=np.expand_dims(X_dev,axis=2)

X_train.shape

(1641, 3139, 1)

In [10]:
X_dev.shape

(197, 3139, 1)

In [21]:
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam

inp = Input((3139,1))

x=Bidirectional(LSTM(16,recurrent_dropout=0.2,return_sequences=True))(inp)
x=Bidirectional(LSTM(32,recurrent_dropout=0.2))(x)
x=Flatten()(x)

x = Dense(256,activation='relu')(x) 

out1 = Dense(21,activation='softmax',name='classes')(x)
out2 = Dense(2,activation='linear',name='indices')(x)

model = Model(inp, [out1,out2])
model.summary()


Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_7 (InputLayer)           [(None, 3139, 1)]    0           []                               
                                                                                                  
 bidirectional_12 (Bidirectiona  (None, 3139, 32)    2304        ['input_7[0][0]']                
 l)                                                                                               
                                                                                                  
 bidirectional_13 (Bidirectiona  (None, 64)          16640       ['bidirectional_12[0][0]']       
 l)                                                                                               
                                                                                            

In [22]:
model.compile(optimizer = 'adam', loss = ['categorical_crossentropy','mse'],metrics=['acc','mse'])

history=model.fit(X_train,[train_labs1, train_labs2],
                  validation_data=([X_dev,[dev_labs1, dev_labs2]]), shuffle=True, epochs=20, batch_size=32)

Epoch 1/20
52/52 [==============================] - 537s 9s/step - loss: 8005.3457 - classes_loss: 2.7880 - indices_loss: 8002.5571 - classes_acc: 0.1926 - classes_mse: 0.0430 - indices_acc: 0.9001 - indices_mse: 8002.5571 - val_loss: 4385.8467 - val_classes_loss: 2.7688 - val_indices_loss: 4383.0781 - val_classes_acc: 0.3452 - val_classes_mse: 0.0405 - val_indices_acc: 0.9239 - val_indices_mse: 4383.0781
Epoch 2/20
52/52 [==============================] - 490s 9s/step - loss: 4558.6147 - classes_loss: 2.4310 - indices_loss: 4556.1846 - classes_acc: 0.3053 - classes_mse: 0.0406 - indices_acc: 0.9129 - indices_mse: 4556.1846 - val_loss: 3875.0039 - val_classes_loss: 2.4041 - val_indices_loss: 3872.6003 - val_classes_acc: 0.3452 - val_classes_mse: 0.0396 - val_indices_acc: 0.9239 - val_indices_mse: 3872.6003
Epoch 3/20
52/52 [==============================] - 476s 9s/step - loss: 4493.9014 - classes_loss: 2.3071 - indices_loss: 4491.5952 - classes_acc: 0.3193 - classes_mse: 0.0400 - indi

KeyboardInterrupt: ignored